In [18]:
import urllib.error
import pandas as pd
from Bio import SeqIO, Entrez
from Bio.Seq import Seq
from openpyxl import workbook, load_workbook
from pydna.dseq import Dseq
from pydna.dseqrecord import Dseqrecord

SsCC9311_VBPO_5_3 = "ATGACGGACCAACCTAAACTTACCGCGCAGCCTGTTCCTGAGGATGCTAACGCCCTGGCAGCCGGCCGAATCAACCCTCCTAATCAGGCTAACGGTGACGAGCAACGCTACGAGTCGGCCAACTACGCGATGTCTTTTACAAAAGGCCTGGACAATAACACCACCACCGGCCTGATTGAGCAGTCCGGTGACTTCGAAGCGTTTCGATCTGCTATTGACAATGGATTCGCTGAAGATTTCACCCGCAACGTCGCGGTACCGCCTGCGGAGCCGCCTCCTAAATTGGAAGCACCGACCGCGGGGACCGTATACGAGTTACAGGGTCCGGATCCGCAGGCAGTTACCATTCCCCCGGCCCCTGCGTTGTGTTCCGACGAACTGACATTTGAGATGGCCGAGGTCTACGAACTGGCCCTGCTGCGCGATCTGCCATTCAACGCCTTCGTCGCGGGGGGTGGATCCGCAGCGTTAGCGGACTCGACCGCCCCTCTCAATTCACTGGCTTACGCTCAGGATGGTTTTAACCGCCCTCCTCGCACCACCAACTCCTCTAATCAGCTGGATGCCCAGACCGTCTTTCGCGGCTCATCTCCGGGGGTTGACCAGGGTCCATACTTATCCCAGTTTATGCTGATCGGTAACGCGTCTCCATCGGAAGGCATCACCCCAGAGCAAGGTTTTATCAACTTCGGTGCCCAACCTATTGATCAGCCTGTACTGGAAGCGCGCCAGCAGGATGATTATATGATGAAATTGGATGACTTGAATCCTGTCCAGCAAGGGTACGAAGTGCGCGCGGATCCTTTTGACCCGTGCAAATCATCTGGTCCGGGCCAGGCATTCACGGGCCAACGCCCTTTCATTAACACGCCGCCTGATCTTGCCACATATGTGAACGTCGATGCCTTATATCAGGCATATCTCAATGCATGCCTGTTACTGCTTGGCAACGGCACTGCCTTCGACCCAGGTTTCGACCTGCTGTCCGGCGGCGGCGAAGGTCTGCTGAATGACCCAGCGGGTGGGCAAAAAGTCCCGCTGAACGCGGGCGGCTTCGCATTATTGGGCGGTCCGAACGTCTTATCTTTGGTTACCGAAGTTGCGACCCGAGGTCTGAAAGCGGTGCCTTACCAGAAATTCAATAACAACTTGCCTCTGCGCCCGGAAGCCCTGGCGGCGCGCATTGAAAAAGCGCAAGAAATTGAATCTCCTTTTCCGACGATTTGTGGCTGTTTTTCTGAAATGGCGTCAGATTTGCAGCAGGTGGTGGATCTGATCCGCAACAACAATCAGTCTCTAGCGGGCGAAGCGACCGCTCTGTTACCGATGGCATTTGCCGAGGGTTCTCCAATGAACCCTGCTTACGGTGCCGGAAATGCGACAGTGGCAGGTGCCTGCGTGACGATTCTGAAAGCGTTTTTCAACACATCCGCGTTGTTTGTGAAAATTAATGACGTGGCCGGCTTCAATTCAAAACAGAACATCCTCGCCCCTCTGAAATGCGGCGATTCAGTGGAAGCGGGTGCCTACCAGGAAACTGACTGCGGCAAACCTCTTGAATTTGAACCTTGCGGATCATTTAACTTGATTGAAGGCAAATATGCTACGTTTAAACCTGACGGGAAAACCAATCAGTCATGTTGCCCACTGACGCTGGAGGGTGAACTGAACAAGCTTGCAGCGAATATTTCTATTGGGCCTAATATGGCGGGTGTGAATTACTTCTCCGATTACTATGACTCTCTGCCTATGGGCGAGGAAATTGCCATAGGGATCTTGGAGGAACAGGCTCTGTGCTATAAAACCGACCCGTTCGTGCTCTCGGTTCCGACGTTTGACGGTGACGTTCGCCCTATTGGTCAACCT"
SsCC9311_VBPO_3_5 = "TACTGCCTGGTTGGATTTGAATGGCGCGTCGGACAAGGACTCCTACGATTGCGGGACCGTCGGCCGGCTTAGTTGGGAGGATTAGTCCGATTGCCACTGCTCGTTGCGATGCTCAGCCGGTTGATGCGCTACAGAAAATGTTTTCCGGACCTGTTATTGTGGTGGTGGCCGGACTAACTCGTCAGGCCACTGAAGCTTCGCAAAGCTAGACGATAACTGTTACCTAAGCGACTTCTAAAGTGGGCGTTGCAGCGCCATGGCGGACGCCTCGGCGGAGGATTTAACCTTCGTGGCTGGCGCCCCTGGCATATGCTCAATGTCCCAGGCCTAGGCGTCCGTCAATGGTAAGGGGGCCGGGGACGCAACACAAGGCTGCTTGACTGTAAACTCTACCGGCTCCAGATGCTTGACCGGGACGACGCGCTAGACGGTAAGTTGCGGAAGCAGCGCCCCCCACCTAGGCGTCGCAATCGCCTGAGCTGGCGGGGAGAGTTAAGTGACCGAATGCGAGTCCTACCAAAATTGGCGGGAGGAGCGTGGTGGTTGAGGAGATTAGTCGACCTACGGGTCTGGCAGAAAGCGCCGAGTAGAGGCCCCCAACTGGTCCCAGGTATGAATAGGGTCAAATACGACTAGCCATTGCGCAGAGGTAGCCTTCCGTAGTGGGGTCTCGTTCCAAAATAGTTGAAGCCACGGGTTGGATAACTAGTCGGACATGACCTTCGCGCGGTCGTCCTACTAATATACTACTTTAACCTACTGAACTTAGGACAGGTCGTTCCCATGCTTCACGCGCGCCTAGGAAAACTGGGCACGTTTAGTAGACCAGGCCCGGTCCGTAAGTGCCCGGTTGCGGGAAAGTAATTGTGCGGCGGACTAGAACGGTGTATACACTTGCAGCTACGGAATATAGTCCGTATAGAGTTACGTACGGACAATGACGAACCGTTGCCGTGACGGAAGCTGGGTCCAAAGCTGGACGACAGGCCGCCGCCGCTTCCAGACGACTTACTGGGTCGCCCACCCGTTTTTCAGGGCGACTTGCGCCCGCCGAAGCGTAATAACCCGCCAGGCTTGCAGAATAGAAACCAATGGCTTCAACGCTGGGCTCCAGACTTTCGCCACGGAATGGTCTTTAAGTTATTGTTGAACGGAGACGCGGGCCTTCGGGACCGCCGCGCGTAACTTTTTCGCGTTCTTTAACTTAGAGGAAAAGGCTGCTAAACACCGACAAAAAGACTTTACCGCAGTCTAAACGTCGTCCACCACCTAGACTAGGCGTTGTTGTTAGTCAGAGATCGCCCGCTTCGCTGGCGAGACAATGGCTACCGTAAACGGCTCCCAAGAGGTTACTTGGGACGAATGCCACGGCCTTTACGCTGTCACCGTCCACGGACGCACTGCTAAGACTTTCGCAAAAAGTTGTGTAGGCGCAACAAACACTTTTAATTACTGCACCGGCCGAAGTTAAGTTTTGTCTTGTAGGAGCGGGGAGACTTTACGCCGCTAAGTCACCTTCGCCCACGGATGGTCCTTTGACTGACGCCGTTTGGAGAACTTAAACTTGGAACGCCTAGTAAATTGAACTAACTTCCGTTTATACGATGCAAATTTGGACTGCCCTTTTGGTTAGTCAGTACAACGGGTGACTGCGACCTCCCACTTGACTTGTTCGAACGTCGCTTATAAAGATAACCCGGATTATACCGCCCACACTTAATGAAGAGGCTAATGATACTGAGAGACGGATACCCGCTCCTTTAACGGTATCCCTAGAACCTCCTTGTCCGAGACACGATATTTTGGCTGGGCAAGCACGAGAGCCAAGGCTGCAAACTGCCACTGCAAGCGGGATAACCAGTTGGA"

SsWH8020_VBPO_5_3 = "ATGACAGACCAGCCTAAACTGACCGCCCAGCCTGTTCGCGAAGACGCCAATGCTCTGGCGGCTGGCCGCGTGCAACCGCAGAACCAAGCCAACGGGGACGAGCAGCGCTACGAGTCCGCATCGTACCCGATGTCGTTCACGAAAGGTCTGGACAATAATACCACGACCGGCCTGGTTGAACAATCCGGCGATTTCGAAGCGTTTCCTTCAGCCATCGACAACGGCTTTGCCGAAGATTTTACTCGCCAGGTTCCTGTTCCAAATGCTGAGCCGCGGCGGAAATTGGAAGCCCCAACCGCTGGGACAGTGTATGAACTGCAGGGACCGGACCCGCAGGCTGTCACTATCCCTCCGGCCCCGGCACTGTGTTCAGATGAACTGACGTTCGAGATCGCAGAAGTCTACGAACTTGCTCTTCTGCGCGATTTACCCTTCAACGCTTTCGTGGCCGGTGGTGGCTCTGCTGCTTTGGCAGACTCAACGGCTCGGCTGAATAACCTGGCGTACGCCCAGGACGGTTTCAACTCTCCTCCACGCAAGACTAACTCTTCAAAGCAGCTGGATGCACAGACAGTGTTTCGGGGTTCTTCTCCTGGTGTAGAACCTGGTCCATACCTGTCGCAGTTTATGCTGATCGGTAACGCGTCTCCGTCGGAAGGCATTACGCCAGATCAGGGCTTCATCAACTTTGGTGCGCAGCCTATCGATCAGCCTGTTCTGGAGGCACCTCAGCAAGACGATTATATGATGAAGTTGGACGACTTGAACCGCGTGGAGCAGGGATATGAGGTTCGCGGCGACCCTTTCGATCCGTGCAAGTCATCCGGTCCGGGCCAGGCGTTTACGGGCCAGCCTCCTTTTATTAACACCCCGCCTGACTTAGCCACTTACGTTAATGTGGACGCGCTGTATCAAGCTTACCTGAACGCGTGCTTACTGTTACTTGGCAATGGTACCCCGTTCGACCCGGGCTTCGATCTGCTGTCTGGCGGCGGCGAAGGCCTGCTAAACGATCCGGCATCCGGTCAAAAGGTGCCGCTGAATGCCGGCGGGTTCGCATTGTTGGGGGGGCCTAACGTACTGTCACTAGTGACCGAAGTGGCAACCCCTGGTCTGAAAGCTGTTCCTTACCAAAAATTCAACAACAACTTGCGCCTGCGCCCGGAAGCACTGGCAGCACGCATTGAGAAAGCGCAGGAGATTGAATCACCTTTTCCTGAAATATGTGGCTGTTTCTCGGAGATGGCTTCAGACCTACAACAAACAGTAGACCTGATTCCTAATAATAACCAATCACTGGCTGGCGATGCGACCGCTTTGCTGCCTATGGCCTTTGCGGAAGGTTCACCGATGAATCCCGCATATGGGGCTGGTAACGCGACGGTTGCGGGCGCCTGTGTCACCATCCTGAAAGCATTCTTCAACACGTCCGCTTTATTTGTAAAAATCAATGACTTGGCAGGCTTTTATTCCAAGCAGAACATTCTCGATCCTCTGAAATGCGGAGACTCTGTCGAAGCAGGTGCGTATCAAGTAACGGATTGTGGTAAACGCCTTGAGTTCGAACGCTGTGGCTCTTTTAACTTGATTGAAGGTAAAGATGCGACCTTTAAACCGGATGGCACTACTAACAAGTCATGCTGCCCCCTTACACTTGAAGGTGAGTTGAATAAGCTGGCGGCAAATATTTCTATCGGTCGCAACATGGCAGGGGTAAACTATTTTTCGGACTATTACGACTCTCTGCCTATGGGCGAGGAGATCGCGATCGGTATCCTCGAGGAGCAGGCGCTTTGCTACAAGACCGACCCCTTTGTTCTGTCCGTCCCGACATTTGACGGTGACGTCCGCCCTATCGGCCAGCCT"
SsWH8020_VBPO_3_5 = "TACTGTCTGGTCGGATTTGACTGGCGGGTCGGACAAGCGCTTCTGCGGTTACGAGACCGCCGACCGGCGCACGTTGGCGTCTTGGTTCGGTTGCCCCTGCTCGTCGCGATGCTCAGGCGTAGCATGGGCTACAGCAAGTGCTTTCCAGACCTGTTATTATGGTGCTGGCCGGACCAACTTGTTAGGCCGCTAAAGCTTCGCAAAGGAAGTCGGTAGCTGTTGCCGAAACGGCTTCTAAAATGAGCGGTCCAAGGACAAGGTTTACGACTCGGCGCCGCCTTTAACCTTCGGGGTTGGCGACCCTGTCACATACTTGACGTCCCTGGCCTGGGCGTCCGACAGTGATAGGGAGGCCGGGGCCGTGACACAAGTCTACTTGACTGCAAGCTCTAGCGTCTTCAGATGCTTGAACGAGAAGACGCGCTAAATGGGAAGTTGCGAAAGCACCGGCCACCACCGAGACGACGAAACCGTCTGAGTTGCCGAGCCGACTTATTGGACCGCATGCGGGTCCTGCCAAAGTTGAGAGGAGGTGCGTTCTGATTGAGAAGTTTCGTCGACCTACGTGTCTGTCACAAAGCCCCAAGAAGAGGACCACATCTTGGACCAGGTATGGACAGCGTCAAATACGACTAGCCATTGCGCAGAGGCAGCCTTCCGTAATGCGGTCTAGTCCCGAAGTAGTTGAAACCACGCGTCGGATAGCTAGTCGGACAAGACCTCCGTGGAGTCGTTCTGCTAATATACTACTTCAACCTGCTGAACTTGGCGCACCTCGTCCCTATACTCCAAGCGCCGCTGGGAAAGCTAGGCACGTTCAGTAGGCCAGGCCCGGTCCGCAAATGCCCGGTCGGAGGAAAATAATTGTGGGGCGGACTGAATCGGTGAATGCAATTACACCTGCGCGACATAGTTCGAATGGACTTGCGCACGAATGACAATGAACCGTTACCATGGGGCAAGCTGGGCCCGAAGCTAGACGACAGACCGCCGCCGCTTCCGGACGATTTGCTAGGCCGTAGGCCAGTTTTCCACGGCGACTTACGGCCGCCCAAGCGTAACAACCCCCCCGGATTGCATGACAGTGATCACTGGCTTCACCGTTGGGGACCAGACTTTCGACAAGGAATGGTTTTTAAGTTGTTGTTGAACGCGGACGCGGGCCTTCGTGACCGTCGTGCGTAACTCTTTCGCGTCCTCTAACTTAGTGGAAAAGGACTTTATACACCGACAAAGAGCCTCTACCGAAGTCTGGATGTTGTTTGTCATCTGGACTAAGGATTATTATTGGTTAGTGACCGACCGCTACGCTGGCGAAACGACGGATACCGGAAACGCCTTCCAAGTGGCTACTTAGGGCGTATACCCCGACCATTGCGCTGCCAACGCCCGCGGACACAGTGGTAGGACTTTCGTAAGAAGTTGTGCAGGCGAAATAAACATTTTTAGTTACTGAACCGTCCGAAAATAAGGTTCGTCTTGTAAGAGCTAGGAGACTTTACGCCTCTGAGACAGCTTCGTCCACGCATAGTTCATTGCCTAACACCATTTGCGGAACTCAAGCTTGCGACACCGAGAAAATTGAACTAACTTCCATTTCTACGCTGGAAATTTGGCCTACCGTGATGATTGTTCAGTACGACGGGGGAATGTGAACTTCCACTCAACTTATTCGACCGCCGTTTATAAAGATAGCCAGCGTTGTACCGTCCCCATTTGATAAAAAGCCTGATAATGCTGAGAGACGGATACCCGCTCCTCTAGCGCTAGCCATAGGAGCTCCTCGTCCGCGAAACGATGTTCTGGCTGGGGAAACAAGACAGGCAGGGCTGTAAACTGCCACTGCAGGCGGGATAGCCGGTCGGA"

def reverse(string):
    string = string[::-1]
    return string

In [12]:
SsCC9311_VBPO = Dseq(SsCC9311_VBPO_5_3, reverse(SsCC9311_VBPO_3_5), ovhg=0, linear=True, circular=False, pos=0)

In [13]:
SsCC9311_VBPO

Dseq(-1866)
ATGA..ACCT
TACT..TGGA

In [19]:
plasmid_5_3 = Seq("ATCCGGATATAGTTCCTCCTTTCAGCAAAAAACCCCTCAAGACCCGTTTAGAGGCCCCAAGGGGTTATGCTAGTTATTGCTCAGCGGTGGCAGCAGCCAACTCAGCTTCCTTTCGGGCTTTGTTTAGCAGCCTAGGTATTAATCAATTAGTGGTGGTGGTGGTGGTGGTGGTGCTCGAGTGCGGCCGCAAGCTTGTCGACGGAGCTCGAATTCGGATCCGATATCGCCATGGTTGAGGAGAAGCCCGGGCTCTTGTCGTCGTCATCGGTACCCAGATCTGGGCTGTCCATGTGCTGGCGTTCGAATTTAGCAGCAGCGGTTTCTTTCATACCAATTGCAGTACTACCGCGTGGCACCAGACCCGCGGAGTGATGGTGATGGTGATGACCAGAACCACTAGTTGATCCTTTTTTCTCGCTTAAGTATTTCACTGTATCAGCATACTGCTGAACAAAAACATCCATATTGCTGGTATCCATACCCTGCGGATTCAGCTGATATTTACCGTTAACAAACATCGCCGGAACGCCACGCAATTGCACGTCAGCTGCAGCTTTTTCCTGCTGAGCGACCAGAGATTTCACCACGAAGCTGTTCCACGCCGCGTCGTACTCTTCACCTTTAATACCTGCGTTGATAAATACATCGCGGATATCAGAAGCAGAACGAATGGTCTGGGTTTTCTGTACGCCTTCAAACAGCGGAACAGTCACTTTGTCTTCCACGCCCAGCGCCATCGCCACAGCCCATGCCTGAGTCAGATCTTTGCCCAGGTCACCACCCATGAAGTTGACGTGGTATTTAGTCATCTTCACGCCTTCCGGCAGTTTTTTCTTCACATTATCAGAAATATGCAGAACTTCTTCAAACTGATAGCAGTGCGGGCAGAAGAAAGAGAAAAACTCCAGCACTTGCGGCGCGCCAGCTACCGGTTTTTCCAGGGTAGTGTACTGTTTACCATCTTCATACTGCGCCGCCGATGCGCTAAACGCTAAAACTAAACCAGCCAGCGCCAGCCAAATCTTTTTCATATGTATATCTCCTTCTTAAAGTTAAACAAAATTATTTCTAGAGGGGAATTGTTATCCGCTCACAATTCCCCTATAGTGAGTCGTATTAATTTCGCGGGATCGAGATCGATCTCGATCCTCTACGCCGGACGCATCGTGGCCGGCATCACCGGCGCCACAGGTGCGGTTGCTGGCGCCTATATCGCCGACATCACCGATGGGGAAGATCGGGCTCGCCACTTCGGGCTCATGAGCGCTTGTTTCGGCGTGGGTATGGTGGCAGGCCCCGTGGCCGGGGGACTGTTGGGCGCCATCTCCTTGCATGCACCATTCCTTGCGGCGGCGGTGCTCAACGGCCTCAACCTACTACTGGGCTGCTTCCTAATGCAGGAGTCGCATAAGGGAGAGCGTCGAGATCCCGGACACCATCGAATGGCGCAAAACCTTTCGCGGTATGGCATGATAGCGCCCGGAAGAGAGTCAATTCAGGGTGGTGAATGTGAAACCAGTAACGTTATACGATGTCGCAGAGTATGCCGGTGTCTCTTATCAGACCGTTTCCCGCGTGGTGAACCAGGCCAGCCACGTTTCTGCGAAAACGCGGGAAAAAGTGGAAGCGGCGATGGCGGAGCTGAATTACATTCCCAACCGCGTGGCACAACAACTGGCGGGCAAACAGTCGTTGCTGATTGGCGTTGCCACCTCCAGTCTGGCCCTGCACGCGCCGTCGCAAATTGTCGCGGCGATTAAATCTCGCGCCGATCAACTGGGTGCCAGCGTGGTGGTGTCGATGGTAGAACGAAGCGGCGTCGAAGCCTGTAAAGCGGCGGTGCACAATCTTCTCGCGCAACGCGTCAGTGGGCTGATCATTAACTATCCGCTGGATGACCAGGATGCCATTGCTGTGGAAGCTGCCTGCACTAATGTTCCGGCGTTATTTCTTGATGTCTCTGACCAGACACCCATCAACAGTATTATTTTCTCCCATGAAGACGGTACGCGACTGGGCGTGGAGCATCTGGTCGCATTGGGTCACCAGCAAATCGCGCTGTTAGCGGGCCCATTAAGTTCTGTCTCGGCGCGTCTGCGTCTGGCTGGCTGGCATAAATATCTCACTCGCAATCAAATTCAGCCGATAGCGGAACGGGAAGGCGACTGGAGTGCCATGTCCGGTTTTCAACAAACCATGCAAATGCTGAATGAGGGCATCGTTCCCACTGCGATGCTGGTTGCCAACGATCAGATGGCGCTGGGCGCAATGCGCGCCATTACCGAGTCCGGGCTGCGCGTTGGTGCGGACATCTCGGTAGTGGGATACGACGATACCGAAGACAGCTCATGTTATATCCCGCCGTTAACCACCATCAAACAGGATTTTCGCCTGCTGGGGCAAACCAGCGTGGACCGCTTGCTGCAACTCTCTCAGGGCCAGGCGGTGAAGGGCAATCAGCTGTTGCCCGTCTCACTGGTGAAAAGAAAAACCACCCTGGCGCCCAATACGCAAACCGCCTCTCCCCGCGCGTTGGCCGATTCATTAATGCAGCTGGCACGACAGGTTTCCCGACTGGAAAGCGGGCAGTGAGCGCAACGCAATTAATGTAAGTTAGCTCACTCATTAGGCACCGGGATCTCGACCGATGCCCTTGAGAGCCTTCAACCCAGTCAGCTCCTTCCGGTGGGCGCGGGGCATGACTATCGTCGCCGCACTTATGACTGTCTTCTTTATCATGCAACTCGTAGGACAGGTGCCGGCAGCGCTCTGGGTCATTTTCGGCGAGGACCGCTTTCGCTGGAGCGCGACGATGATCGGCCTGTCGCTTGCGGTATTCGGAATCTTGCACGCCCTCGCTCAAGCCTTCGTCACTGGTCCCGCCACCAAACGTTTCGGCGAGAAGCAGGCCATTATCGCCGGCATGGCGGCCCCACGGGTGCGCATGATCGTGCTCCTGTCGTTGAGGACCCGGCTAGGCTGGCGGGGTTGCCTTACTGGTTAGCAGAATGAATCACCGATACGCGAGCGAACGTGAAGCGACTGCTGCTGCAAAACGTCTGCGACCTGAGCAACAACATGAATGGTCTTCGGTTTCCGTGTTTCGTAAAGTCTGGAAACGCGGAAGTCAGCGCCCTGCACCATTATGTTCCGGATCTGCATCGCAGGATGCTGCTGGCTACCCTGTGGAACACCTACATCTGTATTAACGAAGCGCTGGCATTGACCCTGAGTGATTTTTCTCTGGTCCCGCCGCATCCATACCGCCAGTTGTTTACCCTCACAACGTTCCAGTAACCGGGCATGTTCATCATCAGTAACCCGTATCGTGAGCATCCTCTCTCGTTTCATCGGTATCATTACCCCCATGAACAGAAATCCCCCTTACACGGAGGCATCAGTGACCAAACAGGAAAAAACCGCCCTTAACATGGCCCGCTTTATCAGAAGCCAGACATTAACGCTTCTGGAGAAACTCAACGAGCTGGACGCGGATGAACAGGCAGACATCTGTGAATCGCTTCACGACCACGCTGATGAGCTTTACCGCAGCTGCCTCGCGCGTTTCGGTGATGACGGTGAAAACCTCTGACACATGCAGCTCCCGGAGACGGTCACAGCTTGTCTGTAAGCGGATGCCGGGAGCAGACAAGCCCGTCAGGGCGCGTCAGCGGGTGTTGGCGGGTGTCGGGGCGCAGCCATGACCCAGTCACGTAGCGATAGCGGAGTGTATACTGGCTTAACTATGCGGCATCAGAGCAGATTGTACTGAGAGTGCACCATATATGCGGTGTGAAATACCGCACAGATGCGTAAGGAGAAAATACCGCATCAGGCGCTCTTCCGCTTCCTCGCTCACTGACTCGCTGCGCTCGGTCGTTCGGCTGCGGCGAGCGGTATCAGCTCACTCAAAGGCGGTAATACGGTTATCCACAGAATCAGGGGATAACGCAGGAAAGAACATGTGAGCAAAAGGCCAGCAAAAGGCCAGGAACCGTAAAAAGGCCGCGTTGCTGGCGTTTTTCCATAGGCTCCGCCCCCCTGACGAGCATCACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCCCCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTCCCTTCGGGAAGCGTGGCGCTTTCTCATAGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGCTCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTATCCGGTAACTATCGTCTTGAGTCCAACCCGGTAAGACACGACTTATCGCCACTGGCAGCAGCCACTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGAGTTCTTGAAGTGGTGGCCTAACTACGGCTACACTAGAAGGACAGTATTTGGTATCTGCGCTCTGCTGAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATCCGGCAAACAAACCACCGCTGGTAGCGGTGGTTTTTTTGTTTGCAAGCAGCAGATTACGCGCAGAAAAAAAGGATCTCAAGAAGATCCTTTGATCTTTTCTACGGGGTCTGACGCTCAGTGGAACGAAAACTCACGTTAAGGGATTTTGGTCATGAACAATAAAACTGTCTGCTTACATAAACAGTAATACAAGGGGTGTTATGAGCCATATTCAACGGGAAACGTCTTGCTCTAGGCCGCGATTAAATTCCAACATGGATGCTGATTTATATGGGTATAAATGGGCTCGCGATAATGTCGGGCAATCAGGTGCGACAATCTATCGATTGTATGGGAAGCCCGATGCGCCAGAGTTGTTTCTGAAACATGGCAAAGGTAGCGTTGCCAATGATGTTACAGATGAGATGGTCAGACTAAACTGGCTGACGGAATTTATGCCTCTTCCGACCATCAAGCATTTTATCCGTACTCCTGATGATGCATGGTTACTCACCACTGCGATCCCCGGGAAAACAGCATTCCAGGTATTAGAAGAATATCCTGATTCAGGTGAAAATATTGTTGATGCGCTGGCAGTGTTCCTGCGCCGGTTGCATTCGATTCCTGTTTGTAATTGTCCTTTTAACAGCGATCGCGTATTTCGTCTCGCTCAGGCGCAATCACGAATGAATAACGGTTTGGTTGATGCGAGTGATTTTGATGACGAGCGTAATGGCTGGCCTGTTGAACAAGTCTGGAAAGAAATGCATAAACTTTTGCCATTCTCACCGGATTCAGTCGTCACTCATGGTGATTTCTCACTTGATAACCTTATTTTTGACGAGGGGAAATTAATAGGTTGTATTGATGTTGGACGAGTCGGAATCGCAGACCGATACCAGGATCTTGCCATCCTATGGAACTGCCTCGGTGAGTTTTCTCCTTCATTACAGAAACGGCTTTTTCAAAAATATGGTATTGATAATCCTGATATGAATAAATTGCAGTTTCATTTGATGCTCGATGAGTTTTTCTAAGAATTAATTCATGAGCGGATACATATTTGAATGTATTTAGAAAAATAAACAAATAGGGGTTCCGCGCACATTTCCCCGAAAAGTGCCACCTGAAATTGTAAACGTTAATATTTTGTTAAAATTCGCGTTAAATTTTTGTTAAATCAGCTCATTTTTTAACCAATAGGCCGAAATCGGCAAAATCCCTTATAAATCAAAAGAATAGACCGAGATAGGGTTGAGTGTTGTTCCAGTTTGGAACAAGAGTCCACTATTAAAGAACGTGGACTCCAACGTCAAAGGGCGAAAAACCGTCTATCAGGGCGATGGCCCACTACGTGAACCATCACCCTAATCAAGTTTTTTGGGGTCGAGGTGCCGTAAAGCACTAAATCGGAACCCTAAAGGGAGCCCCCGATTTAGAGCTTGACGGGGAAAGCCGGCGAACGTGGCGAGAAAGGAAGGGAAGAAAGCGAAAGGAGCGGGCGCTAGGGCGCTGGCAAGTGTAGCGGTCACGCTGCGCGTAACCACCACACCCGCCGCGCTTAATGCGCCGCTACAGGGCGCGTCCCATTCGCCA")
plasmid_3_5 = plasmid_5_3.reverse_complement()

plasmid = Dseqrecord(str(plasmid_5_3), linear=False, circular=True)

In [20]:
plasmid

Dseqrecord(o6106)

In [36]:
from Bio.Restriction import *

In [37]:
linear_plasmid = plasmid.linearize(XbaI)

In [38]:
linear_plasmid

Dseqrecord(-6110)

In [39]:
linear_plasmid.seq

Dseq(-6110)
CTAGAGGG..ATTT    
    TCCC..TAAAGATC